# Adding paths and files

In [140]:
import os
import glob
import polars as pl

# Specify the directory where your CSV files are located
directory = r'D:\github\Cricket-Prediction\data\1_rawData\t20s_csv2'

# Use glob to find all CSV files in the specified directory
info_files = glob.glob(os.path.join(directory, '*_info.csv'))
all_files = glob.glob(os.path.join(directory,'*.csv'))
delivery_files = [file for file in all_files if '_info' not in file]

matches=[]
deliveries=[]
# Print the list of CSV files
for info_file in info_files:
    matches.append(info_file.split('\\')[-1])
for delivery in delivery_files:
    if '_info' not in delivery:
        deliveries.append(delivery.split('\\')[-1])

match_ids=[]
for csv_file in matches:
    match_ids.append(csv_file.split('_')[0])
    

In [ ]:
mathes_list = []
# List to store recalculated match IDs
recalculated_matchids = match_ids[:]
injured_matches = []
import tqdm
import pandas as pd
# Iterate over matches and process each one
for idx, match in enumerate(tqdm.tqdm(matches)):
    try:
        # Load and preprocess the match data
        match = pd.read_csv(os.path.join(directory,f'{match}'), header=None, names=['col1', 'attributes', 'values', 'players', 'code'])
        match = match.drop(columns=['col1', 'players', 'code']).T
        match.columns = match.iloc[0]
        match['match_id'] = match_ids[idx]
        match = match[['match_id','team', 'team', 'gender', 'season', 'winner']].drop('attributes')

        # Reset the index and add match_id as a column
        match = match.reset_index(drop=True)
        mathes_list.append(match)
    except Exception:
        injured_matches.append([match,match_ids[idx]])
matches_data= pd.concat(mathes_list)
matches_data.columns = ['match_id',	'team1',	'team2',	'team1_duplicate', 'team2_duplicate',	'gender', 'season',	'winner']
matches_data = matches_data.drop(columns=['team1_duplicate', 'team2_duplicate'])
matches_data

100%|██████████| 3929/3929 [00:29<00:00, 131.73it/s]


attributes,match_id,team,team,team,team,gender,season,winner
0,1001349,Australia,Sri Lanka,Australia,Sri Lanka,male,2016/17,Sri Lanka
0,1001351,Australia,Sri Lanka,Australia,Sri Lanka,male,2016/17,Sri Lanka
0,1001353,Australia,Sri Lanka,Australia,Sri Lanka,male,2016/17,Australia
0,1004729,Ireland,Hong Kong,Ireland,Hong Kong,male,2016,Hong Kong
0,1007655,Zimbabwe,India,Zimbabwe,India,male,2016,Zimbabwe
...,...,...,...,...,...,...,...,...
0,966763,Pakistan,Sri Lanka,Pakistan,Sri Lanka,male,2015/16,Pakistan
0,966765,Bangladesh,India,Bangladesh,India,male,2015/16,India
0,967081,United Arab Emirates,Netherlands,United Arab Emirates,Netherlands,male,2015/16,Netherlands
0,995467,Sri Lanka,Australia,Sri Lanka,Australia,male,2016,Australia


In [143]:
matches_data.columns = ['match_id',	'team1',	'team2',	'team1_duplicate', 'team2_duplicate',	'gender', 'season',	'winner']
matches_data = matches_data.drop(columns=['team1_duplicate', 'team2_duplicate'])
matches_data

,match_id,team1,team2,gender,season,winner
0,1001349,Australia,Sri Lanka,male,2016/17,Sri Lanka
0,1001351,Australia,Sri Lanka,male,2016/17,Sri Lanka
0,1001353,Australia,Sri Lanka,male,2016/17,Australia
0,1004729,Ireland,Hong Kong,male,2016,Hong Kong
0,1007655,Zimbabwe,India,male,2016,Zimbabwe
...,...,...,...,...,...,...
0,966763,Pakistan,Sri Lanka,male,2015/16,Pakistan
0,966765,Bangladesh,India,male,2015/16,India
0,967081,United Arab Emirates,Netherlands,male,2015/16,Netherlands
0,995467,Sri Lanka,Australia,male,2016,Australia


In [144]:
matches_data

,match_id,team1,team2,gender,season,winner
0,1001349,Australia,Sri Lanka,male,2016/17,Sri Lanka
0,1001351,Australia,Sri Lanka,male,2016/17,Sri Lanka
0,1001353,Australia,Sri Lanka,male,2016/17,Australia
0,1004729,Ireland,Hong Kong,male,2016,Hong Kong
0,1007655,Zimbabwe,India,male,2016,Zimbabwe
...,...,...,...,...,...,...
0,966763,Pakistan,Sri Lanka,male,2015/16,Pakistan
0,966765,Bangladesh,India,male,2015/16,India
0,967081,United Arab Emirates,Netherlands,male,2015/16,Netherlands
0,995467,Sri Lanka,Australia,male,2016,Australia


In [145]:
matches_data = pl.from_pandas(matches_data)

In [146]:
matches_data.null_count()

match_id,team1,team2,gender,season,winner
u32,u32,u32,u32,u32,u32
0,0,0,0,0,0


In [147]:
matches_data.write_csv(os.path.join(directory, '../../2_processedData/matches.csv'))